In [1]:
import os
import sys
sys.path.insert(0, '../')

import openai
import config


openai.api_key  = config.OPENAI_KEY
os.environ['OPENAI_API_KEY'] = config.OPENAI_KEY


In [2]:
LLM_NAME = "gpt-4"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=LLM_NAME, temperature=0)



If you wish to experiment on `LangChain plus platform`:

 * Go to [langchain plus platform](https://www.langchain.plus/) and sign up
 * Create an api key from your account's settings
 * Use this api key in the code below 

In [3]:
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
# os.environ["LANGCHAIN_API_KEY"] = config.LANGCHAIN_KEY

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = '../chroma'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [5]:
# Build prompt
from langchain.prompts import PromptTemplate
# template = """
# The following pieces of texts contain a RITIS tool name follwed by it's description.
# Use these texts to addres the question and give the correct answer. 
# If you don't know the answer, just say that you don't know, don't try to make up an answer. 
# Use four sentences maximum. Keep the answer as concise as possible. 
# {context}
# Question: {question}
# Helpful Answer:"""

template = """
The following pieces of texts contain framgents from RITIS tutorials.
Use these texts to addres the question and give the correct answer. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use ten sentences maximum. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

### Memory

In [6]:
# from langchain.memory import ConversationBufferMemory
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

In [7]:
from langchain.chains import RetrievalQA
question = "What tool can I use to rank bottlenecks?"
qa= RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})



# Create a chatbot that works on your documents

In [8]:
from transformers import pipeline
import gradio as gr

audio_model = pipeline("automatic-speech-recognition")

def transcribe_audio(mic):

    transcription = audio_model(mic)["text"]
    return transcription


def ask_question(question = None, mic = None):
    if mic is None:
        input_text = question
    else:
        input_text = transcribe_audio(mic=mic)
        
    result = qa({"query": input_text})
    
    res = """
    Question: {}\n
    Answer: {}\n
    """.format(input_text, result['result'])
    
    return res

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
with open('../app_description.html', 'r') as f:
    app_description = f.read()

In [11]:
gr.Interface(
    fn=ask_question,
    inputs=[
        gr.inputs.Textbox(label="Question", optional=True),
        gr.Audio(source="microphone", type="filepath", optional=True),
    ],
    outputs="text",
    
    title="RITIS chatbot - v 0.1",  
    description=app_description,
    
    
    examples = [
        ["How can I use congestion scan?", None],
        
        ["What tool can I use to rank bottlenecks?", None],
        ["how can I ask for CO2 emissions in College Park", None],
        ["How to estimate the delay costs", None],
        ["I need to analyze carbon dioxide emissions " \
            "in 2020 at I-95 corridor and check if there " \
            "were any anomalies. Can I do it with RITIS?", None],
    ]
).launch(server_port= 8891)

/tmp/ipykernel_38268/818764809.py:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Question", optional=True),
/tmp/ipykernel_38268/818764809.py:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Question", optional=True),
/tmp/ipykernel_38268/818764809.py:4: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Question", optional=True),
/tmp/ipykernel_38268/818764809.py:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Audio(source="microphone", type="filepath", optional=True),


Running on local URL:  http://127.0.0.1:8891

To create a public link, set `share=True` in `launch()`.
